In [16]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
pd.set_option('display.max_rows', 30)

In [17]:
df = pd.read_csv('./Full_data_after_cleaning.csv', error_bad_lines=False)
df.shape

b'Skipping line 2387: expected 11 fields, saw 37\n'


(4703, 11)

In [18]:
# select numerical columns
df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
# select non-numeric columns
df_non_numeric = df.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values

In [19]:
# % of values missing in each column
values_list = list()
cols_list = list()
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())*100
    cols_list.append(col)
    values_list.append(pct_missing)
pct_missing_df = pd.DataFrame()
pct_missing_df['col'] = cols_list
pct_missing_df['pct_missing'] = values_list

In [20]:
#we will replace missing values with the mode values of that column.
df_non_numeric = df.select_dtypes(exclude=[np.number]) 
non_numeric_cols = df_non_numeric.columns.values
for col in non_numeric_cols:
    missing = df[col].isnull()
    num_missing = np.sum(missing)
    if num_missing > 0:  # impute values only for columns that have missing values
        mod = df[col].describe()['top'] # impute with the most frequently occuring value
        df[col] = df[col].fillna(mod)

In [21]:
df

,Name,Current_Company,Current_Job,Experience,job_location,Last_degree,Graduation_start_year,Graduation_end_year,Last_University,ProfileIDS,skills
0,abrar javed,upwork,professional freelancer,1,pakistan,bs electrical engineering,2015,2020,fast nuces,/in/abrar-javed-49854b14b/,"management,microsoft excel,customer service,st..."
1,mahrukh fatima,devbunch private limited,project coordinator,1,lahore pakistan,bachelor of business administration (bba),2014,2018,fast nuces,/in/mahrukh-fatima-60b863106/,"microsoft office,customer service,microsoft ex..."
2,neha nadeem,daraz,data analyst,1,united states,bachelor of science in computer science,2016,2020,fast nuces,/in/nehanadeem/,"business intelligence (bi),data analysis,data ..."
3,abdul rafeh,no joke marketing,corporate marketing manager,1,united states,master's degree,2017,2018,fast nuces,/in/rafeh11/,"microsoft office,microsoft excel,powerpoint,re..."
4,areeba ayyaz,keeptruckin,software engineer- backend,1,lahore pakistan,bachelor of science in computer science,2016,2020,fast nuces,/in/areeba-ayyaz-6948071a3/,"java,ruby on rails,problem solving,android dev..."
...,...,...,...,...,...,...,...,...,...,...,...
4698,nust uni,fast nuces,software engineer,1,lahore pakistan,bachelor's degree,2016,2016,fast nuces,/in/nust-uni-181a20162/,"c,java,html,programming,software development,o..."
4699,shayan ahmad,playtech,senior java developer,1,"london, england, united kingdom",bachelor of science (bs),2012,2016,fast nuces,/in/yansha/,"c++,c,embedded c,object-oriented programming (..."
4700,hamza sheikh,eers global technologies inc.,advanced embedded developer,2,canada,master of engineering - meng,2018,2019,queen's university - faculty of engineering an...,/in/hamzasheikh876/,"communication,design,front end development,ana..."
4701,fizza surahio,wavetec,software quality assurance intern,1,karachi pakistan,bachelor's degree,2018,2021,fast nuces,/in/fizza-surahio-3143511b3/,"python,data science,machine learning,research,..."


In [25]:
# dropping duplicates by considering all columns other than ProfileIDS
cols_other_than_ProfileIDS = list(df.columns)[-2:]
df.drop_duplicates(subset=cols_other_than_ProfileIDS, inplace=True)
df.head()

,Name,Current_Company,Current_Job,Experience,job_location,Last_degree,Graduation_start_year,Graduation_end_year,Last_University,ProfileIDS,skills
0,abrar javed,upwork,professional freelancer,1,pakistan,bs electrical engineering,2015,2020,fast nuces,/in/abrar-javed-49854b14b/,"management,microsoft excel,customer service,st..."
1,mahrukh fatima,devbunch private limited,project coordinator,1,lahore pakistan,bachelor of business administration (bba),2014,2018,fast nuces,/in/mahrukh-fatima-60b863106/,"microsoft office,customer service,microsoft ex..."
2,neha nadeem,daraz,data analyst,1,united states,bachelor of science in computer science,2016,2020,fast nuces,/in/nehanadeem/,"business intelligence (bi),data analysis,data ..."
3,abdul rafeh,no joke marketing,corporate marketing manager,1,united states,master's degree,2017,2018,fast nuces,/in/rafeh11/,"microsoft office,microsoft excel,powerpoint,re..."
4,areeba ayyaz,keeptruckin,software engineer- backend,1,lahore pakistan,bachelor of science in computer science,2016,2020,fast nuces,/in/areeba-ayyaz-6948071a3/,"java,ruby on rails,problem solving,android dev..."


In [23]:
df.shape

(4703, 11)

In [28]:
df['Name'] = df['Name'].str.lower()
df['Current_Company'] = df['Current_Company'].str.lower()
df['Current_Job'] = df['Current_Job'].str.lower()
df['job_location'] = df['job_location'].str.lower()
df['Last_degree'] = df['Last_degree'].str.lower()
df['Last_University'] = df['Last_University'].str.lower()
df['skills'] = df['skills'].str.lower()


In [29]:
x=df["Current_Job"].unique()
len(x)



1951

In [30]:
# helpful modules
#!pip install fuzzywuzzy
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# set seed for reproducibility
#np.random.seed(0)

C:\Users\basha\.conda\envs\myenv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [31]:
# look at the first ten thousand bytes to guess the character encoding
with open("./Full_data_after_cleaning.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100))

# check what the character encoding might be
print(result)

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [33]:
df = pd.read_csv('./Full_data_after_cleaning.csv', error_bad_lines=False)
df.shape

C:\Users\basha\.conda\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 2387: expected 11 fields, saw 37\n'


(4703, 11)

In [34]:
# get all the unique values in the 'City' column
Current_Job = df['Current_Job'].unique()

# sort them alphabetically and then take a closer look

len(Current_Job)

1951

In [35]:
jobs = ['Software engineer', 'Software Developer',
'freelancer',
'Full-stack developer',
'Front end developer','Back end developer','Blockchain Engineer','Cloud Architect','Java developer','DevOps engineer',
'UX designer',
'Web Developer',
'Security Analyst',
'Project Manager',
'Data scientist',
'data analyst','Data Engineer',
'Business analyst',
'Robotics Engineer',
'Cloud computing engineer',
'Software quality assurance',
'machine learning engineer',
'AI Engineer',
'Mobile App Developer', 
'Game Developer',
'Computer Programmer',
'Database Administrator',
'Computer Network Architect',
'Network Administrator',
'Information Researcher',
]

In [36]:

def find_min_ratio(matches):
    list_of_jobs = []
    for i in matches:
        if i[1] > 70 and i[1] < 100:
            list_of_jobs.append(i[1])
    if len(list_of_jobs) == 0 or len(list_of_jobs) == 1:
        return 0
    else: 
        list_of_jobs = Counter(list_of_jobs)
        common_elements = list_of_jobs.most_common(5)
        last_tuple = min(common_elements,key=lambda item:(-item[1], item[0]))
        min_ratio = last_tuple[0]
        return min_ratio
        

In [37]:
def replace_matches_in_column(matches, string_to_match, df, column, min_ratio = 70):
    strings = df[column].unique()
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    rows_with_matches = df[column].isin(close_matches)

    df.loc[rows_with_matches, column] = string_to_match
    


In [38]:
for i in jobs:
    #print(i)
    matches = fuzzywuzzy.process.extract(i , Current_Job, limit=40, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    min_ratio = find_min_ratio(matches) 
    if(min_ratio == 0):
        continue
    else:
        replace_matches_in_column(matches, i, df, 'Current_Job', min_ratio)


In [39]:
# get all the unique values in the 'City' column
Current_Job = df['Current_Job'].unique()

# sort them alphabetically and then take a closer look

len(Current_Job)

1696

In [41]:
df.to_csv('./Full_data_after_cleaning.csv')
